In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import MDAnalysis as mda
from MDAnalysis.analysis import msd
from scipy.optimize import curve_fit

In [ ]:
mda_traj = mda.Universe("../1-SPCE_343-H2O/topology.pdb", "../1-SPCE_343-H2O/traj.xyz", dt=0.01) # dt=0.01 ps
mda_traj.dimensions = [21.725, 21.725, 21.725, 90, 90, 90] # PBC box : lx ly lz alpha beta gamma

In [ ]:
def all_coordinates_nameO(mda_object):
    """
    Return a 3D array with coordinates of the oxygen atoms all over a trajectory processed via MDAnalysis
    """
    list_atoms = mda_object.atoms.names
    # Define boolean array for efficient selection
    isO = np.array([list_atoms[i] == 'O' for i in range(mda_object.trajectory.n_atoms)])
    nb_O = sum(isO)
    c = np.zeros((mda_object.trajectory.n_frames,nb_O,3))
    for i in range(mda_object.trajectory.n_frames):
        c[i,:,:] = mda_object.trajectory[i].positions[isO]
    return c

def compute_MSD_Owater(mda_object, origin_sep = 100, cell_side = 21.725):
    """ 
    Function for computing the MSD of water oxygen atoms over a trajectory processed via MDAnalysis.\n
    - mda_object: mda.Universe() object containing the trajectory
    - origin_sep: number of time-steps between neigboring time-origins for msd calculation
    - cell side: length of simulation cell side in same units as the trajectory
    """
    # Store O atom coordinates only
    coords = all_coordinates_nameO(mda_object)
    
    # Build array of displacements of every O atom between time-steps
    diffpos = np.diff( coords, axis = 0 ) # for all t (axis = 0), all O (axis = 1), x(t+1)-x(t); y(t+1)-y(t); z(t+1)-z(t) (axis=3)

    # Arrays that will be needed for msd:
    msd_multiorigins = np.zeros(mda_object.trajectory.n_frames)
    norm_msd_multiorigins = np.zeros(mda_object.trajectory.n_frames)
    norm_msd_multiorigins[0] = 1 # prevent division by zero
    
    # Unwrap the trajectory by updating only too large adjacent frame differences:
    diffpos[diffpos > cell_side/2] = diffpos[diffpos > cell_side/2] - cell_side
    diffpos[diffpos < -cell_side/2] = diffpos[diffpos < -cell_side/2] + cell_side
    
    for t0 in range(0, mda_object.trajectory.n_frames, origin_sep):
        displacement_t0 = np.cumsum(diffpos[t0:,:,:],axis=0)    # for all t after t0 (axis = 0), all O (axis = 1), x(t+1)-x(0); y(t+1)-y(0); z(t+1)-z(0) (axis=3)
        sq_displacement_t0 = np.sum(displacement_t0**2,axis=-1) # for all t after t0 (axis = 0), |r(t+1) - r(0)|^2 of all O (axis = 1)
        msd_t0 = np.mean(sq_displacement_t0, axis = 1)          # mean over all O
        msd_multiorigins[1:msd_t0.shape[0]+1] += msd_t0
        norm_msd_multiorigins[1:msd_t0.shape[0]+1] += 1
    
    msd = msd_multiorigins / norm_msd_multiorigins
    return msd


def f(t,d,a):
    """Einstein relation in 3D"""
    return 6*d*t + a


def plot_msd(msd, linear_fit, timestep = 0.01):
    """ Plot MSD with linear fit and display diffusion coefficient """
    time = np.asarray([i*timestep for i in range(len(msd))])
    
    Fig, ax = plt.subplots(1,1, figsize=(12,8))
    ax.plot(time , msd, c='tab:blue', ls='-', lw=2)
    ax.plot(time, f(time, linear_fit[0], linear_fit[1]), c='red', ls='--', lw=1, label=f'D = {linear_fit[0]:.2f} '+r' $\AA{}^2 \cdot ps^{-1}$')
    ax.set_xlabel("Time [ps]", size=18)
    ax.set_ylabel("MSD "+r'[$\AA{}^2$]', size=18)
    ax.legend(prop={'size':18}, loc='best')
    plt.xticks(size=16)
    plt.yticks(size=16)
    plt.grid()
    plt.axhline(y=0, color='k')
    plt.show()
    return None

In [ ]:
msd = compute_MSD_Owater(mda_traj)

timestep = 0.01 # ps
start_fit = 2.0 # ps
start_fit_idx = int(start_fit/timestep)

linear_fit, _ = curve_fit(f, [timestep*i for i in range(len(msd))][start_fit_idx:], msd[start_fit_idx:], p0=[1,20])

plot_msd(msd, linear_fit)